In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
toyota_data = pd.read_csv('/kaggle/input/used-car-dataset-ford-and-mercedes/toyota.csv')

In [ ]:
toyota_data.head()

In [ ]:
toyota_data.isna().sum()

In [ ]:
len(toyota_data)

In [ ]:
toyota_data.info()

In [ ]:
toyota_data.model.value_counts()

In [ ]:
toyota_data.year.value_counts()

In [ ]:
toyota_data['car-age'] = 2020 - toyota_data['year']
toyota_data.drop('year', 1, inplace = True)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.lineplot(x = 'engineSize', y = 'price', data = toyota_data)

In [ ]:
plt.figure(figsize=(10,8))
sns.scatterplot(x= 'mileage',y = 'price', hue = 'fuelType', data = toyota_data)

In [ ]:
plt.figure(figsize=(10,8))
sns.scatterplot(x= 'mpg',y = 'price',hue = 'fuelType', data = toyota_data)

In [ ]:
plt.figure(figsize=(10,8))
sns.scatterplot(x= 'tax',y = 'price', hue = 'fuelType', data = toyota_data)

In [ ]:
plt.figure(figsize=(10,8))
sns.scatterplot(x= 'car-age',y = 'price', hue = 'fuelType', data = toyota_data)

In [ ]:
plt.figure(figsize=(10,8))
sns.catplot(y = 'model', x= 'price' , data = toyota_data)

In [ ]:
plt.figure(figsize = (20,12))
plt.subplot(2,2,1)
sns.boxplot(toyota_data.price)
plt.subplot(2,2,2)
sns.boxplot(toyota_data['tax'])
plt.subplot(2,2,3)
sns.boxplot(toyota_data.mpg)
plt.subplot(2,2,4)
sns.boxplot(toyota_data.engineSize)

In [ ]:
toyota_data.describe()

In [ ]:
print(toyota_data.price.quantile(0.995))
print(toyota_data['tax'].quantile(0.995))
print(toyota_data.engineSize.quantile(0.995))
print(toyota_data['mpg'].quantile(0.995))

In [ ]:
toyota_data = toyota_data[toyota_data.price < toyota_data.price.quantile(0.995)]
toyota_data = toyota_data[(toyota_data['tax'] < toyota_data['tax'].quantile(0.995))& (toyota_data['tax'] > 0)]
toyota_data= toyota_data[(toyota_data.engineSize < toyota_data.engineSize.quantile(0.995)) & (toyota_data.engineSize > 0)]
toyota_data = toyota_data[(toyota_data['mpg'] < toyota_data['mpg'].quantile(0.995)) & (toyota_data['mpg'] > 0)]

In [ ]:
toyota_data.reset_index(drop = True, inplace = True)

In [ ]:
plt.figure(figsize = (20,12))
plt.subplot(2,2,1)
sns.boxplot(toyota_data.price)
plt.subplot(2,2,2)
sns.boxplot(toyota_data['tax'])
plt.subplot(2,2,3)
sns.boxplot(toyota_data.mpg)
plt.subplot(2,2,4)
sns.boxplot(toyota_data.engineSize)

In [ ]:
toyota_data.head()

In [ ]:
toyota_data.info()

In [ ]:
sns.pairplot(data = toyota_data)

In [ ]:
sns.heatmap(toyota_data.corr())

In [ ]:
toyota_data.columns

In [ ]:
cat_cols = ['model', 'transmission', 'fuelType']
toyota_cat = toyota_data[cat_cols]

In [ ]:
toyota_cat = pd.get_dummies(toyota_cat, drop_first = True)
toyota_cat.head()

In [ ]:
num_cols = ['price','mileage', 'tax','mpg', 'engineSize', 'car-age']
toyota_num = toyota_data[num_cols]

In [ ]:
toyota = pd.concat([toyota_num, toyota_cat], axis =1)

In [ ]:
target_variable = toyota['price']
toyota.drop('price',1, inplace = True)

In [ ]:
toyota.head()

In [ ]:
cols = toyota.columns

In [ ]:
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
toyota_pro_data = pd.DataFrame(minmax.fit_transform(toyota), columns = cols)

In [ ]:
toyota_pro_data.head()

In [ ]:
X = pd.concat([toyota_pro_data, target_variable],1)
X

In [ ]:
y = X['price']
X = X.drop('price',1)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20, random_state = 0)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
regression_model = LinearRegression()
regression_model.fit(X_train, y_train)
y_pred = regression_model.predict(X_test)
y_pred

In [ ]:
y_test

In [ ]:
rmse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [ ]:
print(rmse)
print(r2)

In [ ]:
from sklearn.metrics import r2_score
coefficient_of_determination = r2_score(y_test,y_pred, multioutput='variance_weighted')
coefficient_of_determination 

In [ ]:
print("The R2 value is {}". format(round(coefficient_of_determination * 100)))

In [ ]:
import statsmodels.api as sm

In [ ]:
X_train_lm = sm.add_constant(X_train)
lr_1 = sm.OLS(y_train, X_train_lm).fit()
lr_1.params
lr_1.summary()

In [ ]:
lr_1.predict(sm.add_constant(X_test))

In [ ]:
y_test

In [ ]:
print("The R2 value is {}". format(round((r2_score(y_test,y_pred, multioutput='variance_weighted')) * 100)))